In [4]:
!uv pip install matplotlib seaborn

Using Python 3.11.11 environment at: /teamspace/studios/this_studio/.conda
Resolved 15 packages in 125ms                                        
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)--------------     0 B/288.00 KiB          
⠙ Preparing packages... (0/1)-------------- 14.92 KiB/288.00 KiB        
⠙ Preparing packages... (0/1)-------------- 30.92 KiB/288.00 KiB        
⠙ Preparing packages... (0/1)-------------- 46.92 KiB/288.00 KiB        
⠙ Preparing packages... (0/1)-------------- 62.92 KiB/288.00 KiB        
⠙ Preparing packages... (0/1)-------------- 76.71 KiB/288.00 KiB        
⠙ Preparing packages... (0/1)-------------- 92.71 KiB/288.00 KiB        
⠙ Preparing packages... (0/1)-------------- 108.71 KiB/288.00 KiB       
⠙ Preparing packages... (0/1)-------------- 124.71 KiB/288.00 KiB       
⠙ Preparing packages... (0/1)-------------- 140.71 KiB/288.00 KiB       
⠙ Preparing packages... (0/1)m------------- 

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns

df = pd.read_csv('/teamspace/studios/this_studio/Zindi_Clinical_Reasoning_Challenge/data/kenya-clinical-reasoning-challenge20250407-27832-p30dso/cleaned_train.csv')

pd.set_option('display.max_columns', None)  # show all columns

print(df.head())

# print(f"Dataset Shape: {df.shape}")

# print(f"Columns: {df.columns.tolist()}")

# print(df.info())

# print(df.describe(include='all'))

# missing_values = df.isnull().sum()
# missing_values = missing_values[missing_values > 0]
# print("Missing values:\n", missing_values)

# import missingno as msno
# msno.matrix(df)
# plt.show()

# # Check if 'target' column exists
# if 'target' in df.columns:
#     print(df['target'].value_counts())
#     sns.countplot(data=df, x='target')
#     plt.title('Target Variable Distribution')
#     plt.show()

  master_index       county                            health_level  \
0     ID_VBWWP  uasin gishu  sub county hospitals and nursing homes   
1     ID_XMBBY  uasin gishu             national referral hospitals   
2     ID_JZNZW       kiambu  sub county hospitals and nursing homes   
3     ID_QOQTK  uasin gishu             national referral hospitals   
4     ID_ZFJBM  uasin gishu             national referral hospitals   

   years_of_experience                                             prompt  \
0                 18.0  i am a nurse with 18 years of experience in ge...   
1                 17.0  i am a nurse with 17 years of experience in ge...   
2                 12.0  i am a nurse with 12 years of experience in ge...   
3                 12.0  i am a nurse with 12 years of experience in pr...   
4                 16.0  i am a nurse with 16 years of experience in ge...   

          nursing_competency     clinical_panel  \
0  pediatric emergency burns            surgery   
1       

In [2]:
# Separate categorical and numerical features
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"Categorical Columns: {categorical_cols}")
print(f"Numerical Columns: {numerical_cols}")

Categorical Columns: ['master_index', 'county', 'health_level', 'prompt', 'nursing_competency', 'clinical_panel', 'clinician', 'gpt4.0', 'llama', 'gemini', 'ddx_snomed']
Numerical Columns: ['years_of_experience']


In [ ]:
# Histograms for numerical features
df[numerical_cols].hist(figsize=(15, 10), bins=30)
plt.tight_layout()
plt.show()

In [ ]:
# Bar plots for categorical features
for col in categorical_cols:
    plt.figure(figsize=(8, 4))
    sns.countplot(data=df, x=col, order=df[col].value_counts().index)
    plt.title(f'Distribution of {col}')
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
# Correlation matrix to show correlation between numerical variables (bivariate analysis)
corr = df[numerical_cols].corr()

In [ ]:
# Heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
#Outlier Detection
# Boxplots for numerical variables
for col in numerical_cols:
    plt.figure(figsize=(8, 4))
    sns.boxplot(data=df, x=col)
    plt.title(f'Boxplot of {col}')
    plt.show()

In [ ]:
#Feature-Target Relationships
# If 'target' exists
if 'target' in df.columns:
    for col in numerical_cols:
        plt.figure(figsize=(8, 4))
        sns.boxplot(x='target', y=col, data=df)
        plt.title(f'{col} vs Target')
        plt.show()

In [ ]:
#Categorical Features vs Target
# If 'target' exists
if 'target' in df.columns:
    for col in categorical_cols:
        plt.figure(figsize=(8, 4))
        sns.countplot(x=col, hue='target', data=df)
        plt.title(f'{col} vs Target')
        plt.xticks(rotation=45)
        plt.show()

#Summary Observations
# Display some basic summary
print("Summary Observations:")
print("- Dataset shape:", df.shape)
print("- Number of missing values:", missing_values.sum())
print("- Numerical features:", len(numerical_cols))
print("- Categorical features:", len(categorical_cols))
print("- Target distribution (if exists):")
if 'target' in df.columns:
    print(df['target'].value_counts(normalize=True))

In [6]:
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset

# 1. Choose a model and tokenizer
model_name = "distilgpt2"  # Lightweight, good for CPU training
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 2. Add padding token if missing (GPT-2 has none by default)
tokenizer.pad_token = tokenizer.eos_token

# 3. Split into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["prompt"].tolist(),
    df["clinician"].tolist(),
    test_size=0.2,
    random_state=42
)

# 4. Define a custom dataset class
class ClinicalReasoningDataset(Dataset):
    def __init__(self, inputs, targets, tokenizer, max_length=512):
        self.inputs = inputs
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = self.inputs[idx]
        target_text = self.targets[idx]
        encoding = self.tokenizer(
            input_text,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt"
        )
        target_encoding = self.tokenizer(
            target_text,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt"
        )
        input_ids = encoding["input_ids"].squeeze()
        attention_mask = encoding["attention_mask"].squeeze()
        labels = target_encoding["input_ids"].squeeze()

        # Replace padding token id's of the labels by -100 so they are ignored by the loss
        labels[labels == tokenizer.pad_token_id] = -100

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }

# 5. Create datasets
train_dataset = ClinicalReasoningDataset(train_texts, train_labels, tokenizer)
val_dataset = ClinicalReasoningDataset(val_texts, val_labels, tokenizer)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token  # Add pad token for GPT-2 compatibility

In [8]:
import transformers
print(transformers.__version__)

4.52.4


In [9]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments

# 1. Load the base model for causal language modeling (text generation)
model = AutoModelForCausalLM.from_pretrained("distilgpt2")
model.resize_token_embeddings(len(tokenizer))  # Adjust for any added tokens

# 2. Define training arguments (tuned for CPU)
training_args = TrainingArguments(
    output_dir="./gpt2_clinical_model",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=1,
    push_to_hub=False,         # Disable huggingface hub push
    fp16=False,                # CPU only
    report_to="none"
)

# 3. Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

# 4. Start training
trainer.train()

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/tmp/ipykernel_5412/2235743597.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,7.177000,6.984604
2,6.920800,6.905922
3,6.789100,6.896348


TrainOutput(global_step=360, training_loss=7.080420345730252, metrics={'train_runtime': 50.8212, 'train_samples_per_second': 14.167, 'train_steps_per_second': 7.084, 'total_flos': 94066830213120.0, 'train_loss': 7.080420345730252, 'epoch': 3.0})

In [14]:
#Function to generate model outputs
def generate_prediction(prompt, model, tokenizer, max_input_length=200, max_output_length=250):
    # Tokenize the input and truncate if too long
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=max_input_length
    ).to(model.device)

    # Generate response
    output_ids = model.generate(
        **inputs,
        max_length=max_output_length,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        num_return_sequences=1
    )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

In [11]:
#Apply to dataset
df["prediction"] = df["prompt"].apply(lambda x: generate_prediction(x, model, tokenizer))

In [12]:
df["prediction"]

0      i am a nurse with 18 years of experience in ge...
1      i am a nurse with 17 years of experience in ge...
2      i am a nurse with 12 years of experience in ge...
3      i am a nurse with 12 years of experience in pr...
4      i am a nurse with 16 years of experience in ge...
                             ...                        
295    i am a nurse with 12 years of experience in ge...
296    i am a nurse with 22 years of experience in ge...
297    i am a nurse with 7 years of experience in gen...
298    i am a nurse with 15 years of experience in pr...
299    i am a nurse with 18 years of experience in ge...
Name: prediction, Length: 300, dtype: object

In [13]:
submission = df[["master_index", "prediction"]].rename(columns={
    "master_index": "ID",
    "prediction": "Prediction"
})

submission.to_csv("zindi_submission.csv", index=False)

In [ ]:
test_df = pd.read_csv("test.csv")

In [ ]:
test_df["Prediction"] = test_df["Prompt"].apply(lambda x: generate_prediction(x, model, tokenizer))

In [ ]:
submission_df = test_df[["Master_Index", "Prediction"]].rename(columns={
    "Master_Index": "ID"
})
submission_df.to_csv("zindi_submission.csv", index=False)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Load your data
df = pd.read_csv("cleaned_train.csv")

# Use prediction column or fallback
if "prediction" not in df.columns:
    df["prediction"] = df["gpt4.0"]

# BLEU score calculation
smoothie = SmoothingFunction().method4
bleu_scores = [
    sentence_bleu([ref.split()], pred.split(), smoothing_function=smoothie)
    for ref, pred in zip(df["clinician"], df["prediction"])
]

# Compute average BLEU
avg_bleu = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU Score: {round(avg_bleu, 4)}")

In [ ]:
from rouge_score import rouge_scorer

# Load your dataset with ground truth and predictions
df = pd.read_csv("cleaned_train.csv")  # Adjust path if needed

# Use model predictions
if "prediction" not in df.columns:
    df["prediction"] = df["gpt4.0"]  # fallback if custom predictions are not present

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)

# Compute ROUGE-L F1 scores for each sample
rouge_l_scores = [
    scorer.score(ref, pred)["rougeL"].fmeasure
    for ref, pred in zip(df["clinician"], df["prediction"])
]

# Calculate average ROUGE-L score
avg_rouge_l = sum(rouge_l_scores) / len(rouge_l_scores)
print(f"Average ROUGE-L F1 Score: {round(avg_rouge_l, 4)}")